In [13]:
import pickle
import re
import pandas as pd
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
lemmatizer = WordNetLemmatizer()

model = pickle.load(open('model', 'rb'))
vectorizer = pickle.load(open('vectorizer', 'rb'))

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def text_cleaner(X):
    corpus = []
    for i in X:
        review = re.sub(r'\W', ' ', i)
        review = review.lower()
        review = re.sub(r'^br$', ' ', review)
        review = re.sub(r'\s+br\s+',' ',review)
        review = re.sub(r'\s+[a-z]\s+', ' ',review)
        review = re.sub(r'^b\s+', '', review)
        review = re.sub(r'\s+', ' ', review)
        corpus.append(review)
    return X

def sentiment(review):
    sample = [text_cleaner(review)]
    pos_tags = pos_tag(sample)
    sample = [WordNetLemmatizer().lemmatize(text[0], get_wordnet_pos(text[1])) for text in pos_tags]
    sample = vectorizer.transform(sample).toarray()
    sentiment = model.predict(sample)

    if sentiment[0] == 2:
        return ("Positive",max(model.predict_proba(sample)[0]))
    elif sentiment[0] ==1:
        return ("Neutral", max(model.predict_proba(sample)[0]))
    else:
        return ("Negative",max(model.predict_proba(sample)[0]))



In [14]:
positive_review = """We enjoyed our stay at 35th floor garden view!
Staff at the concierge was very accommodating to my request to be at the high floor despite 
surge of guests at that time.We had a great time having breakfast at Spago at 57th floor! 
It was am Amazing experience!"""

neutral_review = """I recently stayed at this hotel I wasn’t impressed the service wasn’t the greatest my 
experience also. Breakfast was good the room average. View was the best thus far. It was close to must sightseeing 
and shopping area metro was not bad"""

negative_review = """Talk about an icon, skyline defining, world renowned, you would imagine the customer 
service is decent, but the front desk actually insulted me and made a mockery out of me, I have never met 
such a rude bunch of hotel staff in my life, in particular a staff named YURI, working the night shift."""


In [15]:
print(sentiment(positive_review))
print(sentiment(neutral_review))
print(sentiment(negative_review))

('Positive', 0.8616891099391982)
('Neutral', 0.7686840237187963)
('Negative', 0.6093531994909682)


In [16]:
df_raw = pd.read_excel('NLP.xlsx', header=None)
df_raw = df_raw.dropna()
df_raw.columns = ['Review', 'Sentiment']

df = df_raw.copy()

pred =[]

for x in df.Review:
    pred.append(sentiment(x))
    
df['Predicted'] = pred
df


,Review,Sentiment,Predicted
0,"Taking an Asian tour, we visited Singapore- wh...",positive,"(Positive, 0.6558534390416442)"
1,We enjoyed our stay at 35th floor garden view!...,positive,"(Positive, 0.8616891099391982)"
2,"Room is spacious, clean and had a great view o...",positive,"(Positive, 0.6838505650177387)"
3,We managed to time this just right as the sun ...,positive,"(Positive, 0.7366090479099145)"
4,We stayed here for the access to the rooftop p...,positive,"(Positive, 0.6170795263462167)"
5,Room is clean and big. Nice view with balcony....,positive,"(Positive, 0.7579978752332803)"
6,Spent five nights in Singapore to acclimatize ...,positive,"(Positive, 0.4731085117510703)"
7,Iconic! Popular despite it’s $300 -800 room ra...,positive,"(Positive, 0.7486068427047716)"
8,Amidst covid-19 with all the travel restrictio...,positive,"(Positive, 0.5328996156531767)"
9,My trip to Marina Bay Sands was absolutely bea...,positive,"(Positive, 0.8947677515348721)"


In [19]:
sentiment('the hotel is not amazing')

('Positive', 0.9631877626290097)

In [ ]:
while True:
    text = input("Please input your review: ")
    print("Your review sentiment is: ",sentiment(text)[0],"\nConfidence percentage of: ", sentiment(text)[1])

Please input your review: good
Your review sentiment is:  Positive 
Confidence percentage of:  0.58752255971321
